# RGB images of all patches around A360 with sources overlaid 

Contact: Céline Combet\
LSST Science Piplines version: Weekly 2025_09\
Container Size: large

For each patch covering a 1 deg field around Abell 360, this notebook:
- generates the RGB image of the patch (using the default settings from DP0.2 tutorials);
- overlays the sources used in the WL analysis, i.e, the red sequence is cut out and a series of WL quality cuts are applied before plotting (this cuts can/should be explored further);
- saves the images as PNG

This uses matplotlib only, but might be more efficient/straightforward using ds9 or firefly. As is, it takes 10-30 sec per image. Also, there is no rendering in the notebook: one should check out the png images that are sequentially created.

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from astropy.wcs import WCS


In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
import lsst.afw.geom as afwGeom

from lsst.afw.image import MultibandExposure
from astropy.visualization import make_lupton_rgb

In [ ]:
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'
#collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0_rc5/DM-49865'

# repo = '/repo/dp1'
# collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
version_str = collection.split('/')
version = version_str[-2:][0]+'_'+version_str[-2:][1]
version

In [ ]:
skymap = butler.get('skyMap', skymap='lsst_cells_v1')

### Create RGB image function
The `create_rgb` function below was pasted from one of the DP0.2 tutorial notebooks

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

### Find tracts and patches for Abell 360

Find all the tracts/patches that falls in a given region around the A360 BCG, and store the results in a dictionary `tp_dict`

In [ ]:
# Position of the BCG for A360
ra_bcg = 37.862 #deg
dec_bcg = 6.98 #deg
delta = 0.5 # deg

# Looking for all patches in delta deg region around it
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
#tp_dict

### Loop over all patches and sequentially load the catalogs (+ apply cuts) and create/save RGB image overlaid with sources.

Currently, the images are saved in an `image_output` folder, with the name `rgb_sourcegal_{version}_{tract}_{patch}.png`. 

In [ ]:
# Get the object catlaog of these patches
obj_num = 0
ra = pd.Series()
dec = pd.Series()

#for tract in list(tp_dict.keys()):
for tract in list(tp_dict.keys())[0:1]: #to check out the first image only
    print(f'Loading objects from tract {tract}, patches:{tp_dict[tract]}')

#    for patch in tp_dict[tract]:
    for patch in tp_dict[tract][0:1]: #to check out the first image only
    
        ######## Load the object catalog ########
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        if 'v29' in version:
            datasetType = 'object_patch' # new naming convention
        else:
            datasetType = 'objectTable'
        obj_cat = butler.get(datasetType, dataId=dataId)
        
        if datasetType == 'object_patch': # in new convention, and obj_cat is now an astropy table. 
        # convert to pandas to leave the rest of the code unchanged. This is inefficient though.
            obj_cat = obj_cat.to_pandas() 
        pattern = r"^g_|^z_"  
        # Drop columns matching the pattern for memory
        obj_cat = obj_cat.drop(columns=[col for col in obj_cat.columns if re.match(pattern, col)])

        # basic cuts
        filt = obj_cat['detect_isPrimary']==True
        filt &= obj_cat['r_cModel_flag']== False
        filt &= obj_cat['i_cModel_flag']== False
        filt &= obj_cat['r_cModelFlux']>0
        filt &= obj_cat['i_cModelFlux']>0
        filt &= obj_cat['refExtendedness'] > 0.5

        obj_cat=obj_cat[filt]

        # red sequence cuts, obtained by eye in the A360 WL notebook on r-i
        mag_i = -2.50 * np.log10(obj_cat['i_cModelFlux']) + 31.4
        mag_r = -2.50 * np.log10(obj_cat['r_cModelFlux']) + 31.4
        # rs_hi = 0.6 - (0.1/5.) * (mag_r-19)
        # rs_low = 0.4 - (0.1/5.)* (mag_r-19)
        rs_hi = 0.64 - (0.1/5.) * (mag_r-19)
        rs_low = 0.44 - (0.1/5.)* (mag_r-19)
        color = mag_r - mag_i
        idx = np.where(np.logical_and(color>rs_low, color<rs_hi))[0]
        idx2 = np.where(mag_r.iloc[idx] < 22)[0] # keep the brightest objects only in RS
        RS_id_list = obj_cat['objectId'].iloc[idx].iloc[idx2]
        obj_cat = obj_cat[~obj_cat['objectId'].isin(RS_id_list)]   

        # lensing quality cuts
        filt = np.sqrt(obj_cat['i_hsmShapeRegauss_e1']**2 + obj_cat['i_hsmShapeRegauss_e2']**2) < 4
        filt &= obj_cat['i_hsmShapeRegauss_sigma']<= 0.4 
        filt &= obj_cat['i_hsmShapeRegauss_flag'] == 0
        filt &= obj_cat['i_blendedness'] < 0.42
        filt &= obj_cat['i_iPSF_flag']==0

        res = 1 - (obj_cat['i_ixxPSF']+ obj_cat['i_iyyPSF']) / (obj_cat['i_ixx']+ obj_cat['i_iyy'])
        filt &= res >= 0.3
        filt &= mag_i <= 24.5
 
        obj_cat=obj_cat[filt]

        ra = pd.concat([ra, obj_cat['coord_ra']], ignore_index=True)
        dec = pd.concat([dec, obj_cat['coord_dec']], ignore_index=True)

        ######### Now make to RGB image of that patch #########
        if 'v29' in version:
            datasetType = 'deep_coadd'
        else:
            datasetType = 'deepCoadd_calexp'
        dataId = {'tract': tract, 'patch' : patch , 'band': 'g', 'skymap':'lsst_cells_v1'}
        coadd_g = butler.get(datasetType, dataId=dataId)
        dataId = {'tract': tract, 'patch' : patch , 'band': 'r', 'skymap':'lsst_cells_v1'}
        coadd_r = butler.get(datasetType, dataId=dataId)
        dataId = {'tract': tract, 'patch' : patch , 'band': 'i', 'skymap':'lsst_cells_v1'}
        coadd_i = butler.get(datasetType, dataId=dataId)
        
        coadds = [coadd_g, coadd_r, coadd_i]
        coadds = MultibandExposure.fromExposures(['g', 'r', 'i'], coadds)


        ###### Plot RGB image, overplot sources, save image as PNG ########
        fig, ax = plt.subplots(figsize=(15, 8), nrows=1, ncols=2, subplot_kw={'projection': WCS(coadd_g.getWcs().getFitsMetadata())})
        
        rgb_original = create_rgb(coadds.image, bgr=['g', 'r', 'i'], scale=None)
        
        rgb_extent = (coadd_g.getBBox().beginX, coadd_g.getBBox().endX,
                      coadd_g.getBBox().beginY, coadd_g.getBBox().endY)
        
        
        ax[0].imshow(rgb_original, origin='lower', extent=rgb_extent)
        
        ax[1].imshow(rgb_original, origin='lower', extent=rgb_extent)
        ax[1].scatter(obj_cat['coord_ra'], obj_cat['coord_dec'],
                   transform=ax[1].get_transform('world'), edgecolor='cyan', facecolor='none',lw=0.5, s=30,
                   label=f'objects')
        
        ax[0].set_xlim([coadd_g.getBBox().beginX, coadd_g.getBBox().endX])
        ax[0].set_ylim([coadd_g.getBBox().beginY, coadd_g.getBBox().endY])
        ax[1].set_xlim([coadd_g.getBBox().beginX, coadd_g.getBBox().endX])
        ax[1].set_ylim([coadd_g.getBBox().beginY, coadd_g.getBBox().endY])
    
        fig.savefig(f'image_output/rgb_sourcegal_{version}_{tract}_{patch}.png')
        plt.close()